In [1]:
# Import packages
import math
import numpy as np
import pickle as pk
import qiskit as qk

# Load IBM backend and provider
token_idx, batch_idx = 5, 23
with open('..\\..\\token_dictionary.p', 'rb') as f:
    token_dict = pk.load(f)
token_values = list(token_dict.values())
token = token_values[token_idx-1]
try:
    qk.IBMQ.disable_account()
except:
    pass
qk.IBMQ.save_account(token=token)
qk.IBMQ.enable_account(token)
provider = qk.IBMQ.get_provider(hub = 'ibm-q')
backend = provider.get_backend('ibmq_5_yorktown') # Yorktown hardware
shots = 8192

qr = qk.QuantumRegister(1)
cr = qk.ClassicalRegister(1)

configrc.store_credentials:WARNING:2021-05-07 11:37:04,741: Credentials already present. Set overwrite=True to overwrite.
C:\Users\kzhang\Anaconda3\lib\site-packages\qiskit\providers\ibmq\ibmqfactory.py:109: UserWarning: Timestamps in IBMQ backend properties, jobs, and job results are all now in local time instead of UTC.
  warnings.warn('Timestamps in IBMQ backend properties, jobs, and job results '


In [2]:
# Create circuits
def makeCirc(gates, gate_list):
    circ = qk.QuantumCircuit(qr,cr)
    for g in range(gates):
        circ.u3(gate_list[g][0], gate_list[g][1], gate_list[g][2], 0)
    circ.measure(0,0)
    return circ

# Return fidelity using counts
def getProb(state, counts, shots):
    if state in counts.keys(): 
        return counts[state]/shots
    else: 
        return 0

In [3]:
# Run circuits
circ_list = pk.load(open('..\\..\\data_york(1_2).p','rb'))[batch_idx]
circ_run = []
for circ in circ_list:
    circ_run.append(makeCirc(circ['gates'], circ['gate_list']))

In [21]:
# Process results
results = qk.execute(circ_run, backend=backend, shots=shots).result()
fidelities = [getProb('0x0', results.data(k)['counts'], shots) for k in range(len(circ_run))]

In [22]:
# Save results
all_circs = pk.load(open('..\\..\\data_york(1_2).p','rb'))
circ_list = all_circs[batch_idx]
for circ_idx in range(len(circ_list)):
    circ_list[circ_idx]['fidelity'] = fidelities[circ_idx]
pk.dump(all_circs, open('..\\..\\data_york(1_2).p','wb'))